# Neural Network

## Perceptrons

In [2]:
from linearalgebra import Vector, dot

In [3]:
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

In [4]:
def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    ''' return 1 if the perceptron "fires",  0 if not'''
    calculation = dot(weights, x) + bias
    return step_function(calculation)

In [5]:
and_weights = [2., 2]
and_bias = -3.

In [6]:
perceptron_output(and_weights, and_bias, [1,1])

1.0

## Feed- forward neural networks

In [7]:
import math

def sigmoid(t: float) -> float:
    return 1/(1+math.exp(-t))

In [8]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    '''weights includes the bias term, inputs includes a 1'''
    return sigmoid(dot(weights, inputs))

In [9]:
from typing import List

In [10]:
def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    '''
    Feeds the inputs vector through the neural network.
    Returns the outputs of all layers (not just the last one).'''
    
    outputs: List[Vector] = []
    
    for layer in neural_network:
        input_with_bias = input_vector + [1]  # Add a constant.
        output = [neuron_output(neuron, input_with_bias)
                  for neuron in layer]
        outputs.append(output)  # add to results.
        #  then the input to the next layers is the output of this one
        
        input_vector = output
        
    return outputs

In [11]:
xor_network = [# hidden layer
                [[20., 20, -30],
                [20., 20, -10]],
                [[-60., 60, -30]]]

In [29]:
feed_forward(xor_network, [1,0])

[[4.5397868702434395e-05, 0.9999546021312976], [0.9999999999999059]]

## Backpropagation

In [21]:
def sqerror_gradients(network: List[List[Vector]],
                      input_vector: Vector,
                      target_vector: Vector) -> List[List[Vector]]:
    """
    Given a neural network, an input vector, and a target vector,
    make a prediction and compute the gradient of the squared error
    loss with respect to the neuron weights.
    """
    # forward pass
    hidden_outputs, outputs = feed_forward(network, input_vector)

    # gradients with respect to output neuron pre-activation outputs
    output_deltas = [output * (1 - output) * (output - target)
                     for output, target in zip(outputs, target_vector)]

    # gradients with respect to output neuron weights
    output_grads = [[output_deltas[i] * hidden_output
                     for hidden_output in hidden_outputs + [1]]
                    for i, output_neuron in enumerate(network[-1])]

    # gradients with respect to hidden neuron pre-activation outputs
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                         dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # gradients with respect to hidden neuron weights
    hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                    for i, hidden_neuron in enumerate(network[0])]

    return [hidden_grads, output_grads]

In [22]:
import random

In [23]:
random.seed(0)

In [24]:
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]]

In [25]:
network = [[[random.random() for _ in range(2+1)],
            [random.random() for _ in range(2+1)],
            [random.random() for _ in range(2+1)]]]

In [26]:
from GradientDescent import gradient_step
import tqdm

In [30]:
learning_rate = 1.0

for epoch in tqdm.trange(20000, desc="neural net for xor"):
    for x, y in zip(xs, ys):
        gradients = sqerror_gradients(network, x, y)
        network = [[gradient_step(neuron, grad, -learning_rate)
                    for neuron, grad in zip(layer, layer_grad)]
                    for layer, layer_grad in zip(network, gradients)]

neural net for xor:   0%|                                                                    | 0/20000 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
x`